# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [7]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [8]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [12]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_PUBLIC_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [10]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [29]:
# Getting the data from the top 10 expensive neighbourhoods
# Group by neightbourhood with no column filters, getting mean data, sorting by average_house_value, and then resetting index
total_avg_house_value = to_data.groupby(
            [to_data.loc[:, "neighbourhood"]]).mean().sort_values("average_house_value", ascending=False).reset_index()
# Remove all but the top 10 neighbourhoods
total_avg_house_value_top10 = total_avg_house_value.iloc[:10]

# Calculate the mean number of dwelling types units per year
# We do not do this at any point in the analysis
# According to the specs it doesnt actually matter what exactly is in the dashboard
# I will use the previously made graphs and follow the given example
# This is the number of dwelling types per year
dwellings_list = ["single_detached_house", "apartment_five_storeys_plus", "movable_dwelling", "semi_detached_house", 
                  "row_house", "duplex", "apartment_five_storeys_less", "other_house"]
# Group by index (year) and specify dwellings
dwelling_units_py = to_data.groupby(to_data.index).sum().loc[:,dwellings_list]

# Calculate the average monthly shelter costs for owned and rented dwellings
shelter_costs = to_data.groupby(to_data.index).mean().loc[:,["shelter_costs_owned", "shelter_costs_rented"]]

### Panel Visualization Functions

In [44]:
# Define Panel visualization functions
def neighbourhood_map():
    mean_values_by_neighbourhood = to_data.groupby([to_data.loc[:, "neighbourhood"]]).mean().reset_index()
    
    mean_neighbour_values_with_locations = pd.concat([df_neighbourhood_locations, mean_values_by_neighbourhood], axis="columns", join="inner")
    
    return px.scatter_mapbox(
                mean_neighbour_values_with_locations,
                lat='lat',
                lon='lon',
                color='average_house_value',
                size='average_house_value',
                zoom=8
)
    
def create_bar_chart(data, title, xlabel, ylabel, color):
    return data.hvplot.bar(
        title=title, 
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color
    ).opts(
        # Rotate x label name 90% for visibility, format y axis to not use scientific format, and increase height so the chart
        # doesn't look squished
        xrotation=90, 
        yformatter="%.0f",
        frame_height = 300)

def create_line_chart(data, title, xlabel, ylabel, color):
    return data.hvplot.line(
        title=title, 
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color
    ).opts(
        # Format Y to not use scientific notation and increase width so the chart doesn't look squished 
        yformatter="%.0f",
        frame_width = 500
    )

def average_house_value():
    avg_house_value = to_data.loc[:, "average_house_value"].groupby(to_data.index).mean()
    return create_line_chart(
        avg_house_value, 
        "Average House Value in Toronto", 
        "Year",
        "Avg House Value",
        "blue"
    )

def average_value_by_neighbourhood():
    avg_house_value_neighbourhood = to_data.loc[:, "average_house_value"].groupby(
                                    [to_data.index, to_data.loc[:, "neighbourhood"]]).mean().reset_index()
    
    return avg_house_value_neighbourhood.hvplot.line(
        x="year",
        y="average_house_value",
        groupby="neighbourhood",
        xlabel="Year", 
        ylabel="Avg House Value", 
        color="blue"
    ).opts(
        # Format Y to not use scientific notation and increase width so the chart doesn't look squished 
        yformatter="%.0f",
        frame_width = 500,
    )

def number_dwelling_types():

    dwelling_type_count_neighbourhood = to_data.loc[:, dwellings_list].groupby(
                                    [to_data.index, to_data.loc[:, "neighbourhood"]]).sum().reset_index()
    year = ["2001", "2006", "2011", "2016"]

    return dwelling_type_count_neighbourhood.hvplot.bar(
        x="year",
        groupby="neighbourhood",
        xlabel="Year", 
        ylabel="Dwelling Type Units"
    ).opts(
        # Format Y to not use scientific notation and increase width so the chart doesn't look squished 
        # Rotate x for visibility
        xrotation = 90,
        yformatter="%.0f",
        frame_width = 600,
        frame_height = 300
    )

def average_house_value_snapshot():
    """filler"""
    # filler

def top_most_expensive_neighbourhoods():
    # Group by neightbourhood with no column filters, getting mean data, sorting by average_house_value, and then resetting index
    total_avg_house_value = to_data.groupby(
                [to_data.loc[:, "neighbourhood"]]).mean().sort_values("average_house_value", ascending=False).reset_index()
    # Remove all but the top 10 neighbourhoods
    total_avg_house_value_top10 = total_avg_house_value.iloc[:10]

    return total_avg_house_value_top10.hvplot.bar(
        x="neighbourhood",
        y="average_house_value",
        xlabel="Neighbourhood", 
        ylabel="Avg House Value",
        title="Top 10 Expensive Neighbourhoods in Toronto"
    ).opts(
        # Format Y to not use scientific notation and increase width so the chart doesn't look squished 
        # Rotate x for visibility
        xrotation = 90,
        yformatter="%.0f",
        frame_width = 600,
        frame_height = 300
    )
def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [47]:
# Create a Title for the Dashboard
dash_title = pn.Column("#A Graphical Analysis of Real Estate in Toronto")

# Define a welcome text
welcome_text = pn.Column("""This is a display of data regarding the quantity and cost of dwellings within Toronto and its 
                            neighbourhood. This data is produced using census data over the 2001-2016 period. The data is
                            organised in tabs, which you can click through at the top of the panel"""
                         , neighbourhood_map())

# Create a tab layout for the dashboard

# First tab: Yearly housing count
# Bar chart for 2001
df_2001 = dwelling_units_py.loc[2001]
bar_2001 = create_bar_chart(df_2001, "Number of dwellings by type - 2001", "Dwelling Type", "Number of Dwellings", "red")

# Bar chart for 2006
df_2006 = dwelling_units_py.loc[2006]
bar_2006 = create_bar_chart(df_2006, "Number of dwellings by type - 2006", "Dwelling Type", "Number of Dwellings", "blue")

# Bar chart for 2011
df_2011 = dwelling_units_py.loc[2011]
bar_2011 = create_bar_chart(df_2011, "Number of dwellings by type - 2011", "Dwelling Type", "Number of Dwellings", "green")

# Bar chart for 2016
df_2016 = dwelling_units_py.loc[2016]
bar_2016 = create_bar_chart(df_2016, "Number of dwellings by type - 2016", "Dwelling Type", "Number of Dwellings", "yellow")

row_1 = pn.Row(bar_2001, bar_2006)
row_2 = pn.Row(bar_2011, bar_2016)
yearly_housing_count = pn.Column(row_1, row_2)

#Second tab: Average house costs analysis
df_owned = shelter_costs.loc[:,"shelter_costs_owned"]
line_owned = create_line_chart(
    df_owned, 
    "Average Monthly Shelter Cost for Owned Dwellings in Toronto", 
    "Year",
    "Avg Monthly Shelter Costs",
    "blue"
)

# Line chart for rented dwellings
df_rented = shelter_costs.loc[:,"shelter_costs_rented"]
line_rented = create_line_chart(
    df_rented, 
    "Average Monthly Shelter Cost for Rented Dwellings in Toronto", 
    "Year",
    "Avg Monthly Shelter Costs",
    "yellow"
)

row_1 = pn.Row(line_owned, line_rented)
avg_house_cost = pn.Column(average_house_value(), row_1)

# Neighbourhood analysis

row_1 = pn.Column(average_value_by_neighbourhood(), number_dwelling_types())
neighbourhood = pn.Row(row_1, top_most_expensive_neighbourhoods())

all_tabs = pn.Tabs(
        ("Welcome", welcome_text),
        ("Yearly Housing Count", yearly_housing_count),
        ("Average Housing Cost", avg_house_cost),
        ("Neighbourhood Analysis", neighbourhood)
)

# Create the main dashboard
dashboard = pn.Column(dash_title, all_tabs)

## Serve the Panel Dashboard

In [48]:
dashboard.servable()

Column
    [0] Column
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Column
            [0] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
            [1] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
        [2] Column
            [0] HoloViews(Curve)
            [1] Row
                [0] HoloViews(Curve)
                [1] HoloViews(Curve)
        [3] Row
            [0] Column
                [0] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
                [1] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
            [1] HoloViews(Bars)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()